<hr style="border-top-width:4px; border-top-color: #2188bf;">
<img align="right" src="images/Logo_Università_Padova.png" alt="Drawing" style="width: 300px"/>
<img align="right" src="images/Logo_Physics_of_Data.jpg" alt="Drawing" style="width: 135px"/>

# MAPD (mod.B) Final Project
# Analysis of Covid-19 papers

### Authors
* [Ardino Rocco](https://github.com/RoccoA97) (University of Padua)
* [Pagano Alice](https://github.com/AlicePagano) (University of Padua)
* [Valente Alessandro](https://github.com/mastrovalentz) (University of Padua)

<hr style="border-top-width:4px; border-top-color: #2188bf;">

<br>
<br>
<br>
<br>
<br>

<hr style="border-top-width:4px; border-top-color: #2188bf;">

# Index <a class="anchor" id="MAPD_B_FP.i"></a>

* 1. [**Introduction**](#MAPD_B_FP.1.)
* 2. [**Word counter distributed algorithm**](#MAPD_B_FP.2.)

<hr style="border-top-width:4px; border-top-color: #2188bf;">

<br>
<br>
<br>
<br>
<br>

<hr style="border-top-width:4px; border-top-color: #2188bf;">

## 1. - Introduction <a class="anchor" id="MAPD_B_FP.1."></a>
* [Index](#MAPD_B_FP.i)

In [49]:
import os
import json
import ast
import string

In [2]:
import dask.bag as db

In [3]:
from dask.distributed import Client, progress
client = Client()

## Load data

In [ ]:
filename = os.path.join('data', 'papers_in_json_singleline', '*.json')
lines = db.read_text(filename)
js = lines.map(json.loads)

In [4]:
# b = db.read_text('data/papers_in_json/*.json').map(json.dumps).map(json.loads)
b = db.read_text('data/papers_in_json_singleline/*.json').map(json.loads)
b

dask.bag<loads, npartitions=1000>

In [6]:
bd = b.to_dataframe()

In [7]:
bd

,paper_id,metadata,abstract,body_text,bib_entries,ref_entries,back_matter
npartitions=1000,,,,,,,
,object,object,object,object,object,object,object
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


## Preprocessing of data

In [8]:
b_body = b.pluck("paper_id")

In [9]:
b_body.take(10, npartitions=10)

('000a0fc8bbef80410199e690191dc3076a290117',
 '000affa746a03f1fe4e3b3ef1a62fdfa9b9ac52a',
 '000b0174f992cb326a891f756d4ae5531f2845f7',
 '000b7d1517ceebb34e1e3e817695b6de03e2fa78',
 '000bc3d4637530496ac4cb798e10d95709eff92d',
 '00a00d0edc750db4a0c299dd1ec0c6871f5a4f24',
 '00a026de6dd835a88da5840c5c3244c3b5721a74',
 '00a0ab182dc01b6c2e737dfae585f050dcf9a7a5',
 '00a2d1c238863845bb309013d9690a04935df5e5',
 '00a3a4742c565f5e2e64ad03fe0f1e16baee89b3')

In [12]:
def merge(record):
    text=''
    for rec in record:
        text+=rec['text']
    return text

texts = b.pluck("body_text").map(merge)
# texts.take(3, npartitions=3)

In [13]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize

In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/rocco/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/rocco/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [22]:
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))

In [50]:
def clean_func_1(text):
    
    text.translate(str.maketrans('', '', string.punctuation))
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    
    return tokens_without_sw

In [95]:
def clean_func(text):

    def merge_text(text,stop_words):
        new_sentence = ''
        for w in text:
            w = w.strip()
            if w.lower() not in stop_words and len(w)>1: 
                new_sentence += w 
                new_sentence += " "
        return new_sentence

    stop_words = set(stopwords.words('english'))
    result = word_tokenize(text)
    result = merge_text(result,stop_words)
    result = tokenizer.tokenize(result)
    result = merge_text(result,stop_words)
    result = word_tokenize(result)
    return result

In [96]:
text_clean = texts.map(clean_func)

In [98]:
#text_clean.take(1)

In [103]:
from collections import Counter

def count_words(text):
    return(dict(zip(Counter(text).keys(), Counter(text).values())))

def order_words(text):
    return {k: v for k, v in sorted(text.items(), key=lambda item: item[1], reverse=True)}

words = text_clean.map(count_words)
words_ordered = words.map(order_words)

In [188]:
#words_ordered.take(1, npartitions=1)

In [190]:
from collections import Counter

# dict_variable = {key:value for (key,value) in dictonary.items()}

def count_words(text):
    counts = dict(zip(Counter(text).keys(), Counter(text).values()))
    wList = [{"word":x , "counts": y} for x,y in counts.items()]
    # wList = [{x : y} for x,y in counts.items()]
    # wList = {x : {"counts" : y} for x,y in counts.items()}
    return wList

def order_words(text):
    return {k: v for k, v in sorted(text.items(), key=lambda item: item[1], reverse=True)}

words = text_clean.map(count_words)
#words_ordered = words.map(order_words)


prova = words#.flatten()#.take(2000)#take(1, npartitions=1)

In [200]:
df = prova.to_dataframe()
df.head(3000)

,0,1,2,3,4,5,6,7,8,9,...,1289,1290,1291,1292,1293,1294,1295,1296,1297,1298
0,"{'word': 'Plasmodium', 'counts': 7}","{'word': 'falciparum', 'counts': 29}","{'word': 'causes', 'counts': 2}","{'word': 'malignant', 'counts': 1}","{'word': 'malaria', 'counts': 6}","{'word': 'cerebral', 'counts': 1}","{'word': 'CM', 'counts': 1}","{'word': 'pregnancy', 'counts': 1}","{'word': 'associated', 'counts': 5}","{'word': 'PAM', 'counts': 1}",...,"{'word': 'positioning', 'counts': 1}","{'word': 'biological', 'counts': 1}","{'word': 'relocating', 'counts': 1}","{'word': 'episomes', 'counts': 1}","{'word': 'regarding', 'counts': 1}","{'word': 'possible', 'counts': 1}","{'word': 'interaction', 'counts': 1}","{'word': 'worthwhile', 'counts': 1}","{'word': 'findings', 'counts': 1}","{'word': 'provide', 'counts': 1}"


In [171]:
from operator import add

words = text_clean.map(count_words)

#def amount_sum (x):
#    return np.sum(np.array([y['amount'] for y in x['transactions']]))
#
#def incr_amount(tot, x):
#    return tot+amount_sum(x)

def incr_amount(tot, x):
    return tot+x['counts']

am = words.foldby('word', binop=incr_amount, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()

#print(sorted(am))

TypeError: list indices must be integers or slices, not str